In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.user_53a9f7f7a254c3731a000070 import mytestvixdata

from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage

from quantopian.pipeline.filters import StaticAssets

my_symbols = StaticAssets(symbols(["AAPL", "VXX"]))

close = USEquityPricing.close

pipe = Pipeline(
    columns={
        'my_dataset': mytestvixdata.vixopen.latest,
        'close':close.latest,
    },
    #screen=mytestvixdata.vixopen.latest.notnull()
    screen=my_symbols,
)
df = run_pipeline(pipe, '2019-08-02', '2019-10-18')
df.tail()

[Quantopian Docs: Pipeline Recipes](https://www.quantopian.com/docs/recipes/pipeline-recipes#pipeline-recipes)
を意訳してみました。ご査収ください。

## Pipeline Recipes

pipleine を使うと、自分が取得したいアセット（銘柄）を指定して、初期化するのが通常の使い方ですが、

なにも指定しなくても初期化はできます。

この場合は、指定された日付と、Quantopianが提供できる**全ての**アセットのマルチインデックスの空DataFrameが返ります


In [ ]:
from quantopian.pipeline import Pipeline
pipe = Pipeline()
df = run_pipeline(pipe, '2019-10-17', '2019-10-18')
df

Pipelineは、`colomns`, `screen`, `domain` のいずれかのオプションを取って、フィルタリング（スクリーニング）できます。

たとえば、`US_EQUITIES` は上場されている全ての米株リストを取得できます。（Quantopianでは米株を扱うことができますので、実質上記のコードと同じ結果になります）


In [ ]:

from quantopian.pipeline.domain import US_EQUITIES

pipe = Pipeline(domain=US_EQUITIES)
df = run_pipeline(pipe, '2019-10-17', '2019-10-18')
df

[QTradableStocksUS](https://www.quantopian.com/docs/api-reference/pipeline-api-reference#quantopian.pipeline.filters.QTradableStocksUS) を使うと、Quantopian がコンテストで指定しているアセットだけを取得できます。（詳しくはリンク先をご覧ください）


In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.pipeline.filters import QTradableStocksUS

qtu = QTradableStocksUS()

# Define a pipeline and filter the output down to the QTradableStocksUS.
pipe = Pipeline(
    screen=qtu
)
df = run_pipeline(pipe, '2019-10-17', '2019-10-18')
df

rows の数が、さっきよりだいぶ減ったのがわかると思います。

### built-in factors を使う

Quantopianでは、様々な built-in factors メソッドが用意されています。
たとえば、Daily ReturnやRSI、VWAPといった通常ユーザーがアセットのデータを使って計算しなくては行けないFactor（とココでは呼びます）も、メソッドと引数（必要であれば）を渡すだけで、全銘柄に対し算出して結果を出力してくれます。どのようなビルトインがあるかは、こちらのレファレンスを参照してください。

[Quantopian Docs: Pipeline Reference built-in-factors](https://www.quantopian.com/docs/api-reference/pipeline-api-reference#built-in-factors)



In [ ]:
from quantopian.pipeline import Pipeline

# 使いたいファクターをインポート
from quantopian.pipeline.factors import DailyReturns, Returns, SimpleBeta

# Daily Return (前日の終値から今日の終値までの変化率)を取得するための関数
# option で、inputs と window_length を渡すことができる。
# 何も渡さない場合は、inputs=[EquityPricing.close], window_length=2
# このようにインスタンス化したものを用意する
daily_returns = DailyReturns()

# Returns は、指定された日数の終値の変化率を返す関数。
# window_lengthという引数で日数を指定
# オプションで、inputsを渡せる。
# 下記の例は、６日間の変化率。
returns_1w = Returns(window_length=6)

# Only allowed_missing_percentage has a default value (0.25). We define target and
# regression_length and override the default value for allowed_missing_percentage.
beta_to_spy = SimpleBeta(
    target=symbols('SPY'),
    regression_length=252,
    allowed_missing_percentage=0.05
)


pipe = Pipeline(
    columns={
        'daily_returns': daily_returns,
        'returns_1w': returns_1w,
        'beta_to_spy': beta_to_spy,
    }
)